In [ ]:
import os
import tensorflow as tf
from datasets import load_dataset, DatasetDict
from transformers import BertTokenizer, TFBertModel, DataCollatorWithPadding
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D, Lambda, Layer, Input, Dropout
from tensorflow.keras.models import Model
import shutil
import pandas as pd
from tqdm.keras import TqdmCallback
import matplotlib.pyplot as plt

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Importing/Loading the training and test dataset

In [ ]:
# path for sets
train_path = '/kaggle/input/lmsys-chatbot-arena/train.csv'
test_path = '/kaggle/input/lmsys-chatbot-arena/test.csv'

# loading datasets
train_dataset = load_dataset('csv', data_files={'train': train_path})['train']
test_dataset = load_dataset('csv', data_files={'test': test_path})['test']

# saving ID
test_ids = test_dataset['id']

In [ ]:
# adding missing columns in the test set
for col in ['model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie']:
    if col not in test_dataset.column_names:
        test_dataset = test_dataset.add_column(col, [""] * len(test_dataset))

# trainsformation to int64
for col in ['winner_model_a', 'winner_model_b', 'winner_tie']:
    train_dataset = train_dataset.map(lambda x: {col: int(x[col]) if x[col] is not None else 0})
    test_dataset = test_dataset.map(lambda x: {col: int(x[col]) if x[col] != "" else 0})

In [ ]:
# using bert-base-cased's files locally
source_dir = '/kaggle/input/huggingface-bert/bert-base-cased'

model_dir = '/kaggle/working/bert-base-cased'
os.makedirs(model_dir, exist_ok=True)

shutil.copy(os.path.join(source_dir, 'config.json'), model_dir)
shutil.copy(os.path.join(source_dir, 'pytorch_model.bin'), model_dir)
shutil.copy(os.path.join(source_dir, 'tf_model.h5'), model_dir)
shutil.copy(os.path.join(source_dir, 'tokenizer.json'), model_dir)
shutil.copy(os.path.join(source_dir, 'vocab.txt'), model_dir)
shutil.copy(os.path.join(source_dir, 'modelcard.json'), model_dir)

In [ ]:
# Tokenization data
tokenizer = BertTokenizer.from_pretrained(model_dir)

def tokenize_function(examples):
    return tokenizer(examples['model_a'], examples['model_b'], padding="max_length", truncation=True, max_length=512)
tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# Convert to tf.data.Dataset with the correct shape
def convert_to_tf_dataset(dataset, label_col=None):
    if label_col:
        dataset = dataset.remove_columns([col for col in dataset.column_names if col != label_col and col not in tokenizer.model_input_names])
    else:
        dataset = dataset.remove_columns([col for col in dataset.column_names if col not in tokenizer.model_input_names])

    return dataset.to_tf_dataset(
        columns=tokenizer.model_input_names,
        label_cols=[label_col] if label_col else None,
        shuffle=True,
        batch_size=64,
        collate_fn=data_collator
    )
# creating tf.data.Dataset for test
def convert_to_tf_dataset_for_inference(dataset):
    dataset = dataset.remove_columns([col for col in dataset.column_names if col not in tokenizer.model_input_names])
    return dataset.to_tf_dataset(
        columns=tokenizer.model_input_names,
        shuffle=False,
        batch_size=16,
        collate_fn=data_collator
    )

In [ ]:
train_dataset = convert_to_tf_dataset(tokenized_datasets, 'winner_model_a')
test_labels = 'winner_model_a'
test_dataset = convert_to_tf_dataset_for_inference(test_tokenized_datasets)


# Check data types and shapes before training
for batch in train_dataset.take(1):
    inputs, labels = batch
    print("Data types:")
    print(f"input_ids: {inputs['input_ids'].dtype}, attention_mask: {inputs['attention_mask'].dtype}, labels: {labels.dtype}")
    print("Data shapes:")
    print(f"input_ids: {inputs['input_ids'].shape}, attention_mask: {inputs['attention_mask'].shape}, labels: {labels.shape}")


In [ ]:
# building a custom model
class BertLayer(Layer):
    def __init__(self, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.bert = TFBertModel.from_pretrained(model_dir, from_pt=True)  # load model from local directory
    
    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state


def create_keras_model():
    input_ids = Input(shape=(512,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(512,), dtype=tf.int32, name='attention_mask')

    bert_output = BertLayer()([input_ids, attention_mask])
    pooled_output = Lambda(lambda x: x[:, 0], output_shape=(768,))(bert_output)
    output = Dense(4, activation='softmax')(pooled_output)

    model = Model(inputs=[input_ids, attention_mask], outputs=output)
    return model

In [ ]:
#learning
with strategy.scope():
    model = create_keras_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=4e-5),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(train_dataset,  epochs=10, callbacks=[TqdmCallback(verbose=2)])


In [ ]:
# prediction
predictions = model.predict(test_dataset)

# creation DataFrame for submission
submission = pd.DataFrame({
    'id': test_ids,
    'winner_model_a': predictions[:, 0],  
    'winner_model_b': predictions[:, 1],
    'winner_model_tie': predictions[:, 2]
})

submission.to_csv('submission.csv', index=False)